In [ ]:
import pandas as pd

In [ ]:
matches = pd.read_csv("matches.csv", index_col=0)

In [ ]:
matches.head()

In [ ]:
matches.shape

In [ ]:
# 2 seasons * 20 squads * 38 matches

2 * 20 * 38

In [ ]:
# Missing Liverpool 2021-2022
matches["team"].value_counts()

In [ ]:
matches[matches["team"] == "Liverpool"].sort_values("date")

In [ ]:
matches["round"].value_counts()

In [ ]:
matches.dtypes

In [ ]:
del matches["comp"]

In [ ]:
del matches["notes"]

In [ ]:
matches["date"] = pd.to_datetime(matches["date"])

In [ ]:
matches["target"] = (matches["result"] == "W").astype("int")

In [ ]:
matches

In [ ]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [ ]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [ ]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [ ]:
matches["day_code"] = matches["date"].dt.dayofweek

In [ ]:
matches

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
train = matches[matches["date"] < '2023-01-01']

In [ ]:
test = matches[matches["date"] > '2023-01-01']

In [ ]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [ ]:
rf.fit(train[predictors], train["target"])

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc = accuracy_score(test["target"], preds)

In [ ]:
acc

In [ ]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [ ]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

In [ ]:
from sklearn.metrics import precision_score

precision_score(test["target"], preds)

In [ ]:
grouped_matches = matches.groupby("team")

In [ ]:
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [ ]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [ ]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
matches_rolling

In [ ]:
matches_rolling = matches_rolling.droplevel('team')

In [ ]:
matches_rolling

In [ ]:
matches_rolling.index = range(matches_rolling.shape[0])

In [ ]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2023-07-01']
    test = data[data["date"] > '2023-07-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
precision